In [1]:
import sys
from awsglue.utils import getResolvedOptions
import pyspark.sql.functions as F
from pyspark import SparkContext
from awsglue.context import GlueContext
from awsglue.dynamicframe import DynamicFrame
from awsglue.job import Job
from pyspark.sql.functions import col, max



Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,None,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:

# from helpers import get_glue_env_var, get_latest_partitions, PARTITION_KEYS
# from helpers import get_glue_env_var, PARTITION_KEYS

glueContext = GlueContext(SparkContext.getOrCreate())
job = Job(glueContext)

PARTITION_KEYS = ["import_year", "import_month", "import_day", "import_date"]

# def get_latest_partitions(dfa):
#    dfa = dfa.where(col('import_year') == dfa.select(max('import_year')).first()[0])
#    dfa = dfa.where(col('import_month') == dfa.select(max('import_month')).first()[0])
#    dfa = dfa.where(col('import_day') == dfa.select(max('import_day')).first()[0])
#    return dfa

def add_import_time_columns(data_frame):
    now = datetime.datetime.now()
    importYear = str(now.year)
    importMonth = str(now.month).zfill(2)
    importDay = str(now.day).zfill(2)
    importDate = importYear + importMonth + importDay

    data_frame = data_frame.withColumn(
        'import_datetime', f.current_timestamp())
    data_frame = data_frame.withColumn(
        'import_timestamp', f.lit(str(now.timestamp())))
    data_frame = data_frame.withColumn('import_year', f.lit(importYear))
    data_frame = data_frame.withColumn('import_month', f.lit(importMonth))
    data_frame = data_frame.withColumn('import_day', f.lit(importDay))
    data_frame = data_frame.withColumn('import_date', f.lit(importDate))
    return data_frame

def add_currency_date_columns(data_frame, currencyYear, currencyMonth, currencyDay):
#     currencyYear = str(date.year)
    currencyMonth = currencyMonth.zfill(2)
    currencyDay = currencyDay.zfill(2)
    currencyDate = currencyYear + currencyMonth + currencyDay

    data_frame = data_frame.withColumn('currency_year', f.lit(currencyYear))
    data_frame = data_frame.withColumn('currency_month', f.lit(currencyMonth))
    data_frame = data_frame.withColumn('currency_day', f.lit(currencyDay))
    data_frame = data_frame.withColumn('currency_date', f.lit(currencyDate))
    return data_frame


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
tableList = ["contacts"]
# tableList = ["public_comments"]
databaseName = "dataplatform-stg-tascomi-raw-zone"


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
daysBuffer = 5
if daysBuffer > 0:
    pushDownPredicate = f"import_date>=date_format(date_sub(current_date, {daysBuffer}), 'yyyyMMdd')"
else:
    pushDownPredicate = ""


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
for nameOfTableToRecast in tableList:
    
#   load data
    source_ddf = glueContext.create_dynamic_frame.from_catalog(
        name_space=databaseName,
        table_name=nameOfTableToRecast,
#         push_down_predicate = pushDownPredicate
        push_down_predicate = "import_date = '20211026'"
    )

    source_df = source_ddf.toDF()
    
#     print((source_df.count()))


#     source_df = get_latest_partitions(source_df)

#   remove json col
#     source_df = source_df.drop("json")
#     source_df = source_df.drop(nameOfTableToRecast)

#   recast
    





FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
## WRITE snapshot TO S3

def add_currency_date_columns(data_frame, currencyYear, currencyMonth, currencyDay):
    currencyMonth = currencyMonth.zfill(2)
    currencyDay = currencyDay.zfill(2)
    currencyDate = currencyYear + currencyMonth + currencyDay

    data_frame = data_frame.withColumn('currency_year', F.lit(currencyYear))
    data_frame = data_frame.withColumn('currency_month', F.lit(currencyMonth))
    data_frame = data_frame.withColumn('currency_day', F.lit(currencyDay))
    data_frame = data_frame.withColumn('currency_date', F.lit(currencyDate))
    return data_frame


PARTITION_KEYS = ["currency_year", "currency_month", "currency_day", "currency_date"]
source_df = add_currency_date_columns(source_df, '2021','10','26')

resultDataFrame = DynamicFrame.fromDF(source_df, glueContext, "resultDataFrame")

target_destination = f"s3://dataplatform-stg-refined-zone/TEMP/snapshot/{nameOfTableToRecast}"
parquetData = glueContext.write_dynamic_frame.from_options(
    frame=resultDataFrame,
    connection_type="s3",
    format="parquet",
    connection_options={"path": target_destination, "partitionKeys": PARTITION_KEYS},
    transformation_ctx="parquetData")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
## WRITE increment TO S3

resultDataFrame = DynamicFrame.fromDF(source_df, glueContext, "resultDataFrame")
target_destination = f"s3://dataplatform-stg-refined-zone/TEMP/increment/{nameOfTableToRecast}"


parquetData = glueContext.write_dynamic_frame.from_options(
    frame=resultDataFrame,
    connection_type="s3",
    format="parquet",
    connection_options={"path": target_destination, "partitionKeys": PARTITION_KEYS},
    transformation_ctx="parquetData")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
def get_latest_snapshot(dfa):
   dfa = dfa.where(col('currency_date') == dfa.select(max('currency_date')).first()[0])
   return dfa

# create snapshot
# load latest snapshot
snapshot_ddf = glueContext.create_dynamic_frame.from_catalog(
        name_space=databaseName,
        table_name=nameOfTableToRecast,
        push_down_predicate = pushDownPredicate
    )

snapshot_df = snapshot_ddf.toDF()
snapshot_df = get_latest_snapshot(snapshot_df)
last_currency_date = snapshot_df.select(max('currency_date')).first()[0]

print(last_currency_date)

# load increments later than the currency date
source_data = glueContext.create_dynamic_frame.from_options(
    connection_type="s3",
    format="parquet",
    pushDownPredicate = f"import_date>{last_currency_date}"
    connection_options={
        "paths": [target_destination],
        "recurse": True
    },
    transformation_ctx="query_addresses"
)
source_df = source_data.toDF()


# apply COU

# add currency date and set it a partition key
PARTITION_KEYS = ["currency_date"]


# write snapshot to S3

